# 2025 세법 개정본 반영

In [6]:
pip install jpype1

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install tabula-py

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install --upgrade langchain chromadb

In [9]:
pip install requests-cache

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from tqdm import tqdm
from tabula import read_pdf
from langchain.document_loaders import PyMuPDFLoader
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.cache import SQLiteCache
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from tabula import read_pdf
from dotenv import load_dotenv

load_dotenv()


# 폴더 경로 설정
pdf_folder = "data/tax_law"

# 폴더 내 PDF 파일 목록 가져오기
pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

# SQLite 캐시 설정
SQLiteCache(".cache_prompt.sqlite")

# 임베딩 모델 초기화
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 모든 PDF 처리
all_docs = []
for pdf_path in pdf_files:
    print(f"Processing {pdf_path}...")

    # Tabula로 PDF에서 표 데이터 읽기
    try:
        tables = read_pdf(pdf_path, pages="all", stream=True, multiple_tables=True)
        table_texts = [table.to_string(index=False, header=True) for table in tables]
    except Exception as e:
        print(f"Tabula Error with {pdf_path}: {e}")
        table_texts = []

    # PyMuPDFLoader로 텍스트 데이터 로드
    try:
        text_loader = PyMuPDFLoader(pdf_path)
        texts = text_loader.load()  # 문서 리스트로 변환
    except Exception as e:
        print(f"PyMuPDFLoader Error with {pdf_path}: {e}")
        texts = []

    # 표 텍스트를 LangChain Document 형식으로 변환
    table_docs = [
        Document(page_content=text, metadata={"source": f"{pdf_path} - Table {i + 1}"})
        for i, text in enumerate(table_texts)
    ]

    # PyMuPDF에서 로드한 텍스트를 LangChain Document 형식으로 변환
    text_docs = [
        Document(page_content=text.page_content, metadata={"source": pdf_path})
        for text in texts
    ]

    # 텍스트 문서와 표 문서를 결합
    all_docs.extend(text_docs + table_docs)

# 텍스트를 청크로 분리
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o",  # 사용하려는 모델 이름
    chunk_size=1000,
    chunk_overlap=50,
)

split_docs = splitter.split_documents(all_docs)

# Vector Store 생성
vector_store = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    collection_name="tax",
    persist_directory="vector_store/chroma/tax"
)

# Retriever 생성
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 10, "fetch_k": 10}
)

Processing data/tax_law\2024_new_tax.pdf...


In [9]:
split_docs[-8]

Document(metadata={'source': 'data/tax_law\\증권거래세법_시행령.pdf'}, page_content='납세의무자의 인적사항, 변경 사유 등을 적은 본점일괄납부 변경신고서를 제출(국세정보통신망을 통한 제출을 포함\n한다)하여야 한다.\n1. 본점 또는 주사무소의 변경\n2. 지점등의 신설ㆍ폐지\n④ 본점 또는 주사무소 관할 세무서장은 제2항에 따라 본점일괄납부를 신고한 자가 다음 각 호의 어느 하나에 해당\n하는 경우를 제외하고는 본점일괄납부 신고를 수리하여야 한다.\n1. 본점일괄납부를 신고한 자가 신고일부터 과거 2년 이내에 「조세범처벌법」에 따라 처벌받은 경우\n2. 본점일괄납부를 신고한 자가 신고일 현재 국세 또는 지방세를 체납하고 있는 경우\n3. 그 밖에 사정의 변경으로 인하여 본점일괄납부가 적당하지 아니한 경우\n⑤ 본점 또는 주사무소 관할 세무서장은 제4항에 따라 본점일괄납부 신고를 수리하거나 거부하는 경우에는 본점일\n괄납부를 신고한 달의 말일까지 신고인에게 그 사실을 알려야 한다.')

In [2]:
# Prompt Template 생성
messages = [
        ("ai", """
        You are an expert assistant. Answer question using only the following context. 
        If you don't know the answer, just say you don't know. Don't make it up. 
        Answer in Korean.
        
        {context}")"""
        ),
        ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser



In [3]:
chain.invoke("친환경자동차의 개별소비세에 대해 변한 사항이 있어? ")

'친환경차의 개별소비세 감면 적용 기한이 2024년 12월 31일에서 2026년 12월 31일까지 연장되었습니다. 또한, 하이브리드차의 개별소비세 감면 한도가 대당 100만 원에서 70만 원으로 축소되었습니다. 이 개정 사항은 2025년 1월 1일 이후 반출하거나 수입 신고하는 분부터 적용됩니다.'

In [4]:
chain.invoke("의료기술협력단을 기부금 손금산입 대상에 대해 변한 사항이 있어? ")


'의료기술협력단을 기부금 손금산입 대상에 포함하는 변경 사항이 있습니다. 법인세법에 따르면, 의료기술협력단은 특례기부금 단체와 일반기부금 단체에 추가되었습니다. 특례기부금 단체에 해당하는 병원이 설립하는 의료기술협력단은 2025년 1월 1일 이후 신고하는 분부터 적용되며, 일반기부금 단체에 해당하는 병원이 설립하는 의료기술협력단은 영 시행일 이후 신고하는 분부터 적용됩니다. 이 개정은 의료기술협력단 설립 및 운영을 지원하기 위한 것입니다.'

In [5]:
chain.invoke("연말정산시 필요한 서류")

'연말정산 시 필요한 서류는 다음과 같습니다:\n\n1. 소득·세액공제 증명서류: 연말정산간소화 서비스를 통해 출력하거나 전산파일로 내려받아야 합니다.\n2. 기부금명세서: 기부금을 공제받기 위해 필요합니다.\n3. 의료비지급명세서: 의료비 공제를 받기 위해 필요합니다.\n4. 신용카드 등 소득공제 신청서: 신용카드 사용에 대한 공제를 받기 위해 필요합니다.\n5. 월세액 세액공제 명세서: 월세에 대한 세액공제를 받기 위해 필요한 서류입니다.\n6. 기타 서류: 난임시술비나 취학 전아동 학원비 등 연말정산간소화 서비스에서 제공되지 않는 자료는 근로자가 직접 수집해야 합니다. \n\n각 서류는 회사에 제출하여 소득·세액공제신고서가 정확히 작성되었는지 확인받아야 합니다.'

In [6]:
chain.invoke("교육세법이란")

'교육세법은 교육의 질적 향상을 도모하기 위해 필요한 교육재정의 확충에 드는 재원을 확보하기 위한 목적으로 제정된 법률입니다.'

In [17]:
chain.invoke("1세대 1주택자 적용 신청방법 알려줘")

'1세대 1주택자의 적용을 신청하려는 납세의무자는 기획재정부령으로 정하는 신청서를 관할세무서장에게 제출해야 합니다.'

In [16]:
chain.invoke("1세대 1주택자 판단 시 주택 수 산정 제외 신청 방법 알려줘")

'죄송하지만 주택 수 산정 제외 신청 방법에 대한 정보는 제공되어 있지 않습니다. 다른 질문이 있으면 말씀해 주세요.'

In [9]:
chain.invoke("연말정산이전 근로소득 원천징수 중점 확인사항 알려줘")

'연말정산 이전에 근로소득 원천징수 시 중점적으로 확인해야 할 사항은 다음과 같습니다:\n\n1. **과세대상 근로소득 포함 항목**: \n   - 파견수당, 자기차량운전보조금, 비과세 한도를 초과한 국외근로소득 등은 근로소득에 포함됩니다.\n   - 현물식사와 별도로 지급받는 식사대, 비과세 아닌 학자금 및 자녀교육비 지원액 등도 근로소득으로 간주됩니다.\n\n2. **이중 근무자 확인**: \n   - 재취직자 또는 2인 이상으로부터 근로소득이 있는 자에 대해 종(전) 근무지의 근로소득이 합산됐는지 확인해야 합니다.\n\n3. **전출·합병·법인전환 등 고용승계자 확인**: \n   - 관계회사 또는 지점 간 전출·입 근로자 등의 연말정산 시 종(전) 근무지의 근로소득이 합산되었는지, 지급명세서 작성 시 근로소득이 구분되었는지 확인해야 합니다.\n\n이러한 사항들을 사전에 확인하여 과다공제에 따른 가산세 부담을 방지하는 것이 중요합니다.'